In [ ]:
# 필요한 라이브러리 설치 (코랩 환경 기준)
# - torch : 파이토치 (딥러닝/행렬 연산)
# - pandas, numpy : 데이터 처리
# - scipy, tqdm : 과학 연산, 진행률 표시
!pip install torch pandas numpy scipy tqdm

# ----- 1. 기본 라이브러리 및 파이토치 관련 모듈 불러오기 -----
import os, zipfile, urllib.request  # 파일/폴더, 압축, 인터넷에서 파일 다운로드
import pandas as pd                 # 테이블 형태 데이터 처리 (DataFrame)
import numpy as np                  # 숫자/배열 연산
import torch                        # 파이토치 메인 모듈
from torch import nn                # 신경망 구현을 위한 모듈 (Linear, Embedding 등)
from torch.utils.data import Dataset, DataLoader  # 파이토치용 데이터셋/배치 로더
from tqdm import tqdm               # 학습 루프 진행 상황 바를 예쁘게 보여주는 라이브러리

# ----- 2. MovieLens 100K 데이터셋 다운로드 및 압축 해제 -----
DATA_DIR = "./data"                 # 데이터를 저장할 폴더 경로
os.makedirs(DATA_DIR, exist_ok=True)  # 폴더가 없으면 새로 만들기

# MovieLens 100K 데이터셋이 있는 URL
url = "http://files.grouplens.org/datasets/movielens/ml-100k.zip"
zip_path = os.path.join(DATA_DIR, "ml-100k.zip")

# 아직 'ml-100k' 폴더가 없다면 = 한 번도 안 받은 상태라면
if not os.path.exists(os.path.join(DATA_DIR, "ml-100k")):
    # zip 파일도 없으면 인터넷에서 zip 파일 다운로드
    if not os.path.exists(zip_path):
        urllib.request.urlretrieve(url, zip_path)
    # zip 파일을 열어서 DATA_DIR 아래에 압축 풀기
    with zipfile.ZipFile(zip_path, "r") as zf:
        zf.extractall(DATA_DIR)

# ----- 3. 평점 데이터(u.data) 불러오기 -----
ratings_path = os.path.join(DATA_DIR, "ml-100k", "u.data")

# u.data 파일 구조:
# userId \t movieId \t rating \t timestamp
ratings = pd.read_csv(
    ratings_path,
    sep="\t",                       # 탭(tab)으로 구분된 파일
    names=["userId", "movieId", "rating", "timestamp"]  # 컬럼 이름 직접 지정
)

# rating 컬럼을 float 타입으로 변환 (계산 편하게)
ratings["rating"] = ratings["rating"].astype(float)

# ----- 4. 영화 메타데이터(u.item) 불러오기 -----
item_path = os.path.join(DATA_DIR, "ml-100k", "u.item")

# u.item 파일은 | 로 구분되어 있고, 컬럼이 매우 많지만
# 여기서는 [0: movieId, 1: title] 두 개만 사용
movies = pd.read_csv(
    item_path,
    sep="|",
    header=None,                    # 헤더가 없는 파일이라 직접 이름 지정
    encoding="latin-1",             # 특수 문자(영어 제목) 처리를 위한 인코딩
    usecols=[0, 1]                  # 0번째(moviedId), 1번째(title) 컬럼만 읽기
)

# 컬럼 이름을 보기 좋게 바꿔주기
movies.columns = ["movieId", "title"]


In [ ]:
ratings.head()

,userId,movieId,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [ ]:
movies.head()

,movieId,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [ ]:
import numpy as np
import pandas as pd
from numpy.linalg import svd   # (이번 단계에선 안 쓰지만, 이하 SVD 실습용으로 import)

# ----- 1. userId, movieId를 정렬해서 '축' 순서를 고정해두기 -----
# 나중에 행렬로 만들 때 행/열 순서가 뒤죽박죽 되면 헷갈리니까,
# 미리 userId와 movieId를 오름차순으로 정렬해 두고,
# 이 순서대로 행렬의 인덱스를 잡아준다.

user_ids = sorted(ratings["userId"].unique())    # 등장한 모든 userId를 모아서 정렬
movie_ids = sorted(ratings["movieId"].unique())  # 등장한 모든 movieId를 모아서 정렬

# ----- 2. 시각화용으로 '일부 유저 / 일부 영화'만 샘플링 -----
# 전체 943명 유저 × 1682개 영화 전체를 한 번에 보여주면 너무 크고 난잡하니까,
# 개념 설명용으로 앞에서 8명, 앞에서 8개만 잘라서 작은 블록을 만든다.

sample_users = user_ids[:8]    # 앞 8명의 userId
sample_movies = movie_ids[:8]  # 앞 8개의 movieId

# ----- 3. userId × movieId 형태의 평점 행렬 만들기 -----
# pivot_table을 쓰면:
#   - index="userId"  → 행이 userId
#   - columns="movieId" → 열이 movieId
#   - values="rating"   → 각 칸에 rating 값이 들어간다.
# 유저가 해당 영화를 평가하지 않은 경우에는 NaN(결측값)으로 남는다.
# 이게 바로 "유저-아이템 평점 행렬" R (sparse matrix)의 기본 형태.

R_sparse = ratings.pivot_table(
    values="rating",
    *****="userId",       # 행 기준
    *******="movieId"     # 열 기준
)

# ----- 4. 그 중에서 우리가 고른 샘플 유저 × 샘플 영화 부분만 잘라오기 -----
# loc를 사용해서 행: sample_users, 열: sample_movies에 해당하는
# 작은 부분 행렬 R_block을 가져온다.
# 이때, 원래 평점이 없는 칸은 그대로 NaN이 유지된다.

R_block = R_sparse.loc[************, ************]    # loc[행 인덱스 리스트, 열 인덱스 리스트]

print("=== 원본 평점 블록 R ===")
display(R_block)


=== 원본 평점 블록 R ===


movieId,1,2,3,4,5,6,7,8
userId,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0
7,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0
8,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN


In [ ]:
# ----- 1. SVD를 위해 NaN을 임시로 채우기 -----
# 지금 R_block에는 "평가 안 한 영화" 자리에 NaN이 들어 있다.
# 그런데 SVD(singular value decomposition)는 NaN을 입력으로 받을 수 없기 때문에,
# 계산을 위해서만 임시로 NaN을 어떤 값으로 채워줘야 한다.

# 여기서는 가장 단순하게:
#   → 전체 데이터의 평점 평균(global_mean)으로 NaN을 채운다.
#   (이건 "모델"이라기보다, SVD를 돌리기 위한 "테크닉"에 가깝다.)

global_mean = ratings["rating"].****()  # 전체 평점의 평균
R_block_filled = R_block.******(global_mean).values  # NaN 값 채우고, (8, 8) dense 행렬로 변환

# ----- 2. SVD 수행: R ≈ U S V^T -----
# numpy.linalg.svd 를 사용해서
#   R_block_filled = U @ np.diag(S) @ Vt
# 형태로 분해한다.
#
# - U  : (8, 8)  → 유저 방향을 기준으로 한 직교 행렬
# - S  : (8,)    → 특이값 벡터 (중요도 순으로 정렬된 스칼라들)
# - Vt : (8, 8)  → 아이템 방향을 기준으로 한 직교 행렬의 전치

U, S, Vt = svd(R_block_filled, *************=False) # full_matrices 인자 설정

# ----- 3. rank-2 근사 만들기 (중요도 높은 2개 축만 남기기) -----
# SVD의 강점은, 상위 k개의 특이값과 대응하는 축만 사용해서
# 원래 행렬을 "차원이 낮은" 형태로 근사할 수 있다는 점이다.
#
# 여기서는 k=2로 잡아서:
#   - U_k:  상위 2개 잠재요인에 대한 유저 표현  (8 × 2)
#   - S_k:  상위 2개 특이값 (2 × 2 대각행렬)
#   - V_k:  상위 2개 잠재요인에 대한 아이템 표현 (8 × 2)
# 을 만든다.

k = 2  # rank-2

U_k = U[:, :k]             # (8, 2)
S_k = np.diag(S[:k])       # (2, 2)
V_k = Vt[:k, :].T          # (8, 2)  (Vt의 상위 2행을 꺼내서 전치)

# ----- 4. P, Q 정의 (유저/아이템 잠재벡터) -----
# 행렬분해 관점에서는 보통
#   R ≈ P @ Q^T
# 형태로 많이 표현한다.
#
# 여기서:
#   - P: 유저 잠재요인 행렬 (user × k)
#   - Q: 아이템 잠재요인 행렬 (item × k)
#
# SVD 결과를 이 형태로 맞추면:
#   P = U_k @ S_k
#   Q = V_k
# 라고 둘 수 있다.
# 이렇게 하면 (P @ Q^T)가 rank-2 근사 행렬이 된다.

P = U_k @ S_k   # (8, 2)  user latent
Q = V_k         # (8, 2)  item latent

# ----- 5. 보기 좋게 DataFrame으로 변환해서 출력 -----
# 각 유저 / 영화가 잠재요인 f1, f2 상에서 어떤 좌표를 갖는지 확인할 수 있다.
# 숫자 자체보다는:
#   "유저/영화를 2차원 잠재공간 위 점으로 표현한다"
# 는 개념이 중요하다.

P_df = pd.DataFrame(P, index=sample_users, columns=["f1", "f2"])
Q_df = pd.DataFrame(Q, index=sample_movies, columns=["f1", "f2"])

print("=== P 행렬 (user × 2 latent) ===")
display(P_df.round(3))

print("=== Q 행렬 (movie × 2 latent) ===")
display(Q_df.round(3))


=== P 행렬 (user × 2 latent) ===


,f1,f2
1,-9.988,3.170
2,-10.155,-0.053
3,-9.976,-0.240
4,-9.976,-0.240
5,-9.978,-0.018
6,-9.783,-0.506
7,-11.523,-1.607
8,-9.791,-0.275


=== Q 행렬 (movie × 2 latent) ===


,f1,f2
1,-0.382,0.398
2,-0.334,-0.065
3,-0.353,0.176
4,-0.361,-0.247
5,-0.341,-0.066
6,-0.365,0.419
7,-0.349,0.066
8,-0.342,-0.749


In [ ]:
# ----- 1. P, Q를 다시 곱해서 원본 행렬을 '재구성' -----
# 앞에서 만든 P, Q는 rank-2 잠재공간에서 유저/아이템을 표현한 거였다.
# 이제 이 둘을 다시 곱하면:
#   R_hat_block = P @ Q.T  ≈  R_block_filled
#
# 즉, "유저 × 아이템" 평점 행렬을 근사한 결과가 나온다.
# 이 R_hat_block에는 더 이상 NaN이 없고, 모든 칸에 예측값이 채워져 있다.

R_hat_block = P @ Q.T   # (8, 8)

# 보기 편하게 DataFrame으로 변환
R_hat_df = pd.DataFrame(
    R_hat_block,
    index=************,   # 행: userId
    columns=************ # 열: movieId
)

# ----- 2. 관측된 칸(원래 평점이 있던 칸)에서만 예측값 비교 -----
# R_block : 원본 평점 (관측된 곳만 숫자, 나머지는 NaN)
# R_hat_df: rank-2 MF로 재구성된 예측값 (모든 칸 숫자)
#
# 관측된 칸에 대해서:
#   "원래 값 vs 예측 값" 이 얼마나 비슷한지 보는 게 목적이다.
#
# 그래서, R_block이 NaN인 곳은 예측도 NaN으로 가리고,
# R_block이 숫자인 곳만 R_hat_df 값을 남긴다.

observed_pred = R_hat_df.where(~R_block.isna(), np.nan)

print("=== 관측된 칸에 대한 R_hat (원래 값 vs 예측 비교용) ===")
print("원본 R (observed 부분)")
display(R_block)

print("예측 R_hat (같은 위치, 관측된 곳만)")
display(observed_pred.round(2))

# ----- 3. '실제로 쓰고 싶은' 완성 행렬 만들기 -----
# 추천 시스템 관점에서 진짜 하고 싶은 일은:
#   - 이미 평점이 있는 칸은 '그대로' 두고,
#   - 아직 평점이 없는 칸(NaN)만 '예측값'으로 채우는 것.
#
# 즉, 관측된 데이터는 존중하고,
# 빈 칸만 모델이 채워 넣는 형태의 완성 행렬을 만드는 것이다.
#
# 판다스 where을 사용해서:
#   - 조건(~R_block.isna())인 칸 → R_block 그대로 유지
#   - 그 외(NaN이었던 곳)        → R_hat_df 값으로 대체

completed_block = R_block.where(~R_block.isna(), R_hat_df)

print("=== NaN만 예측으로 채워서 완성한 행렬 (observed는 원본 값 유지) ===")
display(completed_block.round(2))


=== 관측된 칸에 대한 R_hat (원래 값 vs 예측 비교용) ===
원본 R (observed 부분)


movieId,1,2,3,4,5,6,7,8
userId,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
6,4.0,NaN,NaN,NaN,NaN,NaN,2.0,4.0
7,NaN,NaN,NaN,5.0,NaN,NaN,5.0,5.0
8,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN


예측 R_hat (같은 위치, 관측된 곳만)


,1,2,3,4,5,6,7,8
1,5.07,3.13,4.08,2.82,3.19,4.97,3.69,1.04
2,3.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3.80,3.34,NaN,NaN,NaN,NaN,NaN,NaN
6,3.53,NaN,NaN,NaN,NaN,NaN,3.38,3.73
7,NaN,NaN,NaN,4.56,NaN,NaN,3.91,5.15
8,NaN,NaN,NaN,NaN,NaN,NaN,3.40,NaN


=== NaN만 예측으로 채워서 완성한 행렬 (observed는 원본 값 유지) ===


movieId,1,2,3,4,5,6,7,8
userId,,,,,,,,
1,5.00,3.00,4.00,3.00,3.00,5.00,4.00,1.00
2,4.00,3.40,3.57,3.68,3.46,3.68,3.54,3.52
3,3.71,3.35,3.48,3.66,3.41,3.54,3.46,3.60
4,3.71,3.35,3.48,3.66,3.41,3.54,3.46,3.60
5,4.00,3.00,3.52,3.61,3.40,3.63,3.48,3.43
6,4.00,3.30,3.36,3.66,3.36,3.36,2.00,4.00
7,3.76,3.95,3.78,5.00,4.03,3.53,5.00,5.00
8,3.63,3.29,3.40,3.60,3.35,3.46,3.00,3.56


In [ ]:
# ===== 셀 1: user/movie 인덱스 매핑 + train/val/test split =====

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# 1) userId, movieId 전체 범위 확인
# - MovieLens의 userId/movieId는 1,2,3,... 식이지만
#   중간에 빠진 숫자가 있을 수 있다.
# - 우리는 "0 ~ (N-1)" 형태의 연속된 번호를 만들어
#   Embedding 레이어에서 인덱스로 쓰기 쉽게 만든다.
user_ids = sorted(ratings["userId"].unique())
movie_ids = sorted(ratings["movieId"].unique())

num_users = len(user_ids)
num_items = len(movie_ids)
print("num_users:", num_users, "num_items:", num_items)

# 2) 실제 ID → 0부터 시작하는 내부 인덱스 만들기
#   예) user_ids = [1, 3, 7] 이라면
#       user2idx = {1:0, 3:1, 7:2}
# - 모델은 이 "내부 인덱스"만 사용하고,
#   나중에 추천 결과를 해석할 때 다시 실제 ID로 매핑하면 된다.
user2idx = {u: i for i, u in enumerate(user_ids)}
item2idx = {m: i for i, m in enumerate(movie_ids)}

# ratings DataFrame에 내부 인덱스 컬럼 추가
ratings["user_idx"] = ratings["userId"].map(user2idx)
ratings["item_idx"] = ratings["movieId"].map(item2idx)

# 3) 학습에 꼭 필요한 컬럼만 남기기
# - user_idx : 유저 인덱스 (0 ~ num_users-1)
# - item_idx : 영화 인덱스 (0 ~ num_items-1)
# - rating   : 실제 평점 (float)
# timestamp 등은 여기서는 사용하지 않으므로 제거
df_idx = ratings[["user_idx", "item_idx", "rating"]]

# 4) train / val / test로 8:1:1 분할
#   (1) 먼저 train vs temp를 80:20으로 나누고
train_df, temp_df = train_test_*****(
    df_idx,
    test_size=0.2,
    random_state=42
)

#   (2) temp를 다시 val / test로 50:50 나누기 → 최종 80/10/10
val_df, test_df = train_test_*****(
    temp_df,
    test_size=0.5,
    random_state=42
)

print(len(train_df), len(val_df), len(test_df))

num_users: 943 num_items: 1682
80000 10000 10000


In [ ]:
# ===== 셀 2: PyTorch Dataset / DataLoader =====

import torch
from torch.utils.data import Dataset, DataLoader

# 1) 평점 데이터를 PyTorch용 Dataset 클래스로 포장하기
# --------------------------------------------------
# PyTorch에서 모델을 학습할 때는 보통:
#   - Dataset  : "샘플 1개"를 어떻게 꺼낼지 정의
#   - DataLoader: Dataset에서 여러 개를 모아 배치(batch)를 만들어 주는 도우미
#
# 우리의 한 샘플은 (user_idx, item_idx, rating) 한 줄이라고 볼 수 있다.
#   예) user_idx=10, item_idx=50, rating=4.0
#
# 그래서 Dataset이 __getitem__(idx)에서 이 3가지를 튜플로 돌려주도록 만든다.

class RatingDataset(Dataset):
    def __init__(self, df):
        # df: train_df, val_df, test_df 중 하나
        #     (컬럼: user_idx, item_idx, rating)
        self.users = df["****_***"].values                     # numpy array
        self.items = df["****_***"].values                     # numpy array
        self.ratings = df["r*****"].values.astype("float32")   # 파이토치용 float32로 변환

    def __len__(self):
        # 전체 샘플 개수 (len(dataset) 호출 시 사용)
        return len(self.ratings)

    def __getitem__(self, idx):
        # idx번째 샘플 하나를 꺼내서 반환
        # → DataLoader가 이걸 여러 번 호출해서 미니배치를 만든다.
        return (
            self.users[idx],    # user_idx (int)
            self.items[idx],    # item_idx (int)
            self.ratings[idx]   # rating   (float32)
        )


# 2) train / val / test용 Dataset 인스턴스 생성
# ---------------------------------------------
train_dataset = RatingDataset(train_df)
val_dataset   = RatingDataset(val_df)
test_dataset  = RatingDataset(test_df)

# 3) Dataset을 DataLoader로 감싸서 배치 단위로 사용
# --------------------------------------------------
# DataLoader 역할:
#   - Dataset에서 __getitem__을 반복 호출해서
#   - batch_size만큼 묶어서 (배치 텐서로) 모델에 전달
#   - (학습용이면) 매 epoch마다 데이터를 섞어줄 수도 있음 (shuffle=True)
#
# 여기 구성:
#   - train_loader: 학습용 → shuffle=True (에폭마다 샘플 순서 섞기)
#   - val_loader  : 검증용 → shuffle=False (그대로)
#   - test_loader : 최종 평가용 → shuffle=False

train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)
val_loader   = DataLoader(val_dataset,   batch_size=1024, shuffle=False)
test_loader  = DataLoader(test_dataset,  batch_size=1024, shuffle=False)

len(train_dataset), len(val_dataset), len(test_dataset)

(80000, 10000, 10000)

In [ ]:
# ===== 셀 3: MF 모델 정의 (biased MF) + Adam optimizer =====

import torch.nn as nn

# GPU(CUDA)를 쓸 수 있으면 CUDA, 아니면 CPU 사용
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

class MF(nn.Module):
    """
    Matrix Factorization (MF) 모델
    - user, item을 잠재공간(latent space) 상의 벡터로 표현하고
    - 두 벡터의 내적(dot product) + bias 로 평점을 예측하는 모델
    - SVD에서 봤던: R ≈ P @ Q^T 의 '학습 버전'이라고 보면 된다.
    """
    def __init__(self, num_users, num_items, n_factors=100):
        super().__init__()

        # ----- 1. 유저 / 아이템 임베딩 (P, Q 역할) -----
        # user_emb: 각 user_idx에 대해 길이 n_factors짜리 벡터를 하나씩 학습
        # item_emb: 각 item_idx에 대해 길이 n_factors짜리 벡터를 하나씩 학습
        #
        # → 결국 user_emb.weight 가 P (num_users × n_factors),
        #         item_emb.weight 가 Q (num_items × n_factors)에 대응한다고 볼 수 있다.
        self.user_emb = nn.Embedding(***_*****, *_********)
        self.item_emb = nn.Embedding(***_*****, *_********)

        # ----- 2. 유저 / 아이템 별 bias -----
        # 어떤 유저는 항상 점수를 짜게 줄 수도 있고, 후하게 줄 수도 있다.
        # 어떤 아이템은 전체적으로 평점이 높거나, 낮을 수도 있다.
        #
        # 그래서 user_bias, item_bias를 따로 두어서:
        #   예측 = (u · v) + user_bias + item_bias + global_bias
        # 형태로 '기본 성향'을 보정해준다.
        self.user_bias = nn.Embedding(num_users, 1)
        self.item_bias = nn.Embedding(num_items, 1)

        # ----- 3. 전체 평균 bias (스칼라 하나) -----
        # 전체 데이터의 평균 평점 정도를 전역 bias로 두고,
        # 나머지를 user/item bias와 latent factor들이 담당하게 하는 구조.
        self.global_bias = nn.Parameter(torch.zeros(1))

        # ----- 4. 파라미터 초기화 -----
        # 임베딩 벡터는 너무 큰 값으로 시작하면 학습 초기에 폭주할 수 있어서
        # 평균 0, 표준편차 0.01인 작은 정규분포로 초기화한다.
        nn.init.normal_(self.user_emb.weight, std=0.01)
        nn.init.normal_(self.item_emb.weight, std=0.01)
        # bias는 처음엔 0에서 시작
        nn.init.zeros_(self.user_bias.weight)
        nn.init.zeros_(self.item_bias.weight)

    def forward(self, user_idx, item_idx):
        """
        user_idx: (batch_size,)  유저 인덱스 텐서
        item_idx: (batch_size,)  아이템 인덱스 텐서

        반환값: (batch_size,)  예측 평점
        """
        # 1) 각 유저 / 아이템의 임베딩 벡터 뽑기  → (B, k)
        u = self.user_emb(user_idx)        # 유저 잠재벡터
        v = self.item_emb(item_idx)        # 아이템 잠재벡터

        # 2) 두 벡터의 element-wise 곱 후, 차원별 합 = dot product  → (B,)
        dot = (u * v).sum(dim=1)

        # 3) user, item bias를 인덱스로 조회해서 가져오기  → (B, 1) → squeeze로 (B,)
        bu = self.user_bias(user_idx).squeeze()
        bi = self.item_bias(item_idx).squeeze()

        # 4) 최종 예측 = 내적 + 유저 bias + 아이템 bias + 전역 bias
        #    (전역 bias는 파라미터 하나라서 shape broadcasting으로 모든 배치에 더해진다.)
        return dot + bu + bi + self.global_bias  # (B,)

# ----- 5. MF 모델 인스턴스 만들기 -----
model = MF(num_users, num_items, n_factors=100).to(device)

# 손실 함수: 평균제곱오차(MSE)
criterion = nn.MSELoss()

# ----- 6. global bias를 train set 평균 평점으로 초기화 -----
# 처음부터 전역 bias를 0으로 두는 것보다,
# train 데이터의 평균 평점(예: 3.5점)으로 맞춰두면
# 초기 예측이 훨씬 안정적이다.
with torch.no_grad():
    model.global_bias.fill_(train_df["rating"].mean())

# ----- 7. Adam optimizer 설정 -----
# - lr: 학습률
# - weight_decay: L2 정규화 (너무 큰 파라미터가 되지 않게 약간의 패널티)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=0.005,
    weight_decay=1e-5
)

model  # 모델 구조 출력 (레이어 / 파라미터 수 등 확인용)


device: cuda


MF(
  (user_emb): Embedding(943, 100)
  (item_emb): Embedding(1682, 100)
  (user_bias): Embedding(943, 1)
  (item_bias): Embedding(1682, 1)
)

In [ ]:
# ===== 셀 4: 평가 함수 (RMSE) =====

from math import sqrt

def evaluate(loader):
    """
    주어진 DataLoader(test_loader 등)에 대해 RMSE를 계산하는 함수.

    - loader 안에는 (user_idx, item_idx, rating) 배치들이 들어 있음.
    - 모델이 예측한 평점과 실제 rating의 차이를 이용해
      Root Mean Squared Error(RMSE)를 계산한다.
    - 예측값은 1~5 범위를 벗어날 수 있으므로, 평가 전에 1~5로 잘라(clamp) 준다.
    """
    model.eval()          # 평가 모드 (dropout, batchnorm 등이 있다면 비활성화)
    total_se = 0.0        # squared error 누적합
    count = 0             # 전체 평점 개수

    with torch.no_grad(): # 평가 시에는 gradient 계산 불필요 (메모리/속도 절약)
        for u, i, r in loader:
            # 1) 배치를 device(GPU/CPU)로 옮기고, 타입 맞추기
            u = u.to(device).long()   # user 인덱스 (정수형)
            i = i.to(device).long()   # item 인덱스 (정수형)
            r = r.to(device)          # 실제 평점 (float)

            # 2) MF 모델로 예측 평점 계산
            pred = model(u, i)        # (batch_size,)

            # 3) 평점 범위(1~5)를 벗어나는 예측을 잘라내기
            #    예: 5.7 -> 5.0, 0.8 -> 1.0
            pred = torch.clamp(pred, 1.0, 5.0)

            # 4) (예측 - 실제)의 제곱을 모두 더하기
            total_se += ((pred - r) ** 2).sum().item()
            count += len(r)

    # 5) RMSE = sqrt( MSE )
    return sqrt(total_se / count)

In [ ]:
# ===== 셀 5: MF 학습 루프 (SGD) + Early Stopping + 최종 test 평가 =====

from math import sqrt

n_epochs = 100

# Early Stopping용 변수들
best_val = float("inf")   # 지금까지 본 val RMSE 중 가장 작은 값
best_state = None         # 그때의 모델 파라미터
best_epoch = None         # 그때의 epoch 번호

for epoch in range(1, n_epochs + 1):
    model.train()
    total_se = 0.0   # train RMSE 계산용 (sum of squared errors)

    # ----- 1. train_loader로 한 epoch 학습 -----
    for u, i, r in train_loader:
        # 배치 데이터를 GPU/CPU로 옮기고 타입 맞춰주기
        u = u.to(device).****()   # user index long 타입
        i = i.to(device).****()   # item index도 long 타입
        r = r.to(device)          # rating (float tensor)

        # 매 배치마다 gradient 초기화
        optimizer.****_****()  #기존 gradient 비우기 (zero_grad)

        # 모델 forward → 예측 평점
        pred = model(u, i)

        # MSE 손실 계산 (학습 시에는 클램프 X)
        loss = criterion(pred, r)

        # 역전파로 gradient 계산
        loss.********() #역전파 수행 함수

        # 파라미터 업데이트 (SGD/Adam 등)
        optimizer.step()

        # train RMSE 계산을 위해 제곱오차 누적
        total_se += ((pred - r) ** 2).sum().item()

    # 한 epoch 끝난 뒤, 전체 train 데이터 기준 RMSE 계산
    train_rmse = sqrt(total_se / len(train_dataset))

    # ----- 2. 검증용 val RMSE 계산 -----
    # 여기서는 모델을 업데이트하지 않고, 성능만 본다.
    # (evaluate 함수 내부에서 model.eval() + no_grad() 사용 중)
    val_rmse = ********(val_loader) #위에서 만든 평가 함수 호출

    print(
        f"Epoch {epoch:03d} | "
        f"train RMSE: {train_rmse:.4f} | val RMSE: {val_rmse:.4f}"
    )

    # ----- 3. Early Stopping: val RMSE가 가장 좋았던 순간 저장 -----
    # test가 아니라 "val"을 기준으로 best 모델을 고른다는 점이 중요하다.
    if val_rmse < best_val:
        best_val = val_rmse
        best_epoch = epoch
        # state_dict()는 파라미터 텐서들을 담고 있는 dict
        # .cpu().clone()으로 안전하게 복사해 둔다.
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}

print("\nBest val RMSE:", best_val, "at epoch", best_epoch)

# ----- 4. 학습이 모두 끝난 뒤, 가장 성능 좋았던 시점으로 롤백 -----
model.load_state_dict(best_state)

# ----- 5. 진짜 test 성능은 여기서 '딱 한 번'만 측정 -----
# → test 데이터는 모델 선택에 사용되지 않고,
#   최종 성능 보고용으로만 쓰이게 된다.
test_rmse = evaluate(test_loader)
print("Final test RMSE:", test_rmse)


Epoch 001 | train RMSE: 1.0631 | val RMSE: 0.9758
Epoch 002 | train RMSE: 0.8777 | val RMSE: 0.9186
Epoch 003 | train RMSE: 0.7128 | val RMSE: 0.9239
Epoch 004 | train RMSE: 0.5598 | val RMSE: 0.9463
Epoch 005 | train RMSE: 0.4450 | val RMSE: 0.9667
Epoch 006 | train RMSE: 0.3635 | val RMSE: 0.9809
Epoch 007 | train RMSE: 0.3053 | val RMSE: 0.9928
Epoch 008 | train RMSE: 0.2618 | val RMSE: 1.0015
Epoch 009 | train RMSE: 0.2279 | val RMSE: 1.0099
Epoch 010 | train RMSE: 0.2024 | val RMSE: 1.0162
Epoch 011 | train RMSE: 0.1815 | val RMSE: 1.0219
Epoch 012 | train RMSE: 0.1656 | val RMSE: 1.0272
Epoch 013 | train RMSE: 0.1528 | val RMSE: 1.0311
Epoch 014 | train RMSE: 0.1428 | val RMSE: 1.0342
Epoch 015 | train RMSE: 0.1345 | val RMSE: 1.0379
Epoch 016 | train RMSE: 0.1286 | val RMSE: 1.0394
Epoch 017 | train RMSE: 0.1235 | val RMSE: 1.0424
Epoch 018 | train RMSE: 0.1217 | val RMSE: 1.0436
Epoch 019 | train RMSE: 0.1202 | val RMSE: 1.0455
Epoch 020 | train RMSE: 0.1187 | val RMSE: 1.0456


In [ ]:
# ===== 셀 6 (옵션): 원본 + NaN만 채운 MF 완성 행렬 =====

# ----- 1. userId × movieId 기준의 '원본 평점 행렬' 다시 만들기 -----
# 앞에서는 (user_idx, item_idx)로 모델을 학습했지만,
# 사람이 보기에는 원래의 userId / movieId 기준이 더 직관적이다.
# pivot_table을 써서:
#   - 행: userId
#   - 열: movieId
#   - 값: rating
# 형태의 sparse 행렬(빈 칸은 NaN)을 만든다.

R_sparse_full = ratings.pivot_table(
    values="rating",
    index="userId",
    columns="movieId"
)

# ----- 2. 모든 (유저, 아이템) 쌍에 대해 예측값 R_hat_full 계산 -----
# 지금까지 학습한 MF 모델은 (user_idx, item_idx)를 넣으면 평점을 예측해 준다.
# 여기서는:
#   - 전체 유저  num_users
#   - 전체 아이템 num_items
# 에 대해 한 번에 "모든 조합 (u, i)"의 예측을 만들어서
# (num_users × num_items) 행렬 R_hat_full로 만든다.

with torch.**_****():  # 예측 단계에서는 gradient 불필요
    # 0, 1, ..., num_users-1
    user_idx_tensor = torch.******(num_users, device=device) #0~num_users-1 정수 시퀀스(arange)
    # 0, 1, ..., num_items-1
    item_idx_tensor = torch.******(num_items, device=device) #0~num_users-1 정수 시퀀스(arange)

    # U_grid: 각 행이 한 유저, 각 열이 아이템 인덱스와 매칭되도록
    #   예: num_users=3, num_items=4라면
    #   U_grid =
    #     [[0, 0, 0, 0],
    #      [1, 1, 1, 1],
    #      [2, 2, 2, 2]]
    U_grid = user_idx_tensor.********(1).******(1, num_items)  # unsqueeze + repeat로 (num_users, num_items)

    # I_grid: 각 열이 한 아이템, 각 행이 유저 인덱스와 매칭되도록
    #   I_grid =
    #     [[0, 1, 2, 3],
    #      [0, 1, 2, 3],
    #      [0, 1, 2, 3]]
    I_grid = item_idx_tensor.********(0).******(num_users, 1)  # unsqueeze + repeat로 (num_users, num_items)

    # 모델은 1차원 벡터 (batch,) 형태로 인덱스를 받으니,
    # 2D grid를 일렬로 펼쳐서 (num_users * num_items,) 형태로 만든다.
    U_flat = U_grid.reshape(-1)  # 길이: num_users * num_items
    I_flat = I_grid.reshape(-1)

    # 한 번에 모든 (u, i)에 대해 예측 수행
    preds_flat = model(U_flat, I_flat)
    # 평점 범위 [1, 5]로 잘라주기 (모델이 조금 튀었더라도 실제 평점 범위로 제한)
    preds_flat = torch.clamp(preds_flat, 1.0, 5.0)

    # 다시 (num_users, num_items) 모양의 행렬로 reshape
    R_hat_full = preds_flat.view(num_users, num_items).cpu().numpy()

# ----- 3. 예측 행렬을 DataFrame으로 변환 (userId / movieId 기준) -----
R_hat_df_full = pd.DataFrame(
    R_hat_full,
    index=user_ids,    # 행: 실제 userId
    columns=movie_ids  # 열: 실제 movieId
)

# ----- 4. "NaN만 예측으로 채운 완성 행렬" 만들기 -----
# R_sparse_full : 원본 평점 (관측된 곳만 숫자, 나머지는 NaN)
# R_hat_df_full : MF 모델이 예측한 '모든' 평점
#
# 우리가 원하는 건:
#   - 원래 평점이 있던 칸은 그대로 두고 (신뢰할 수 있는 관측값)
#   - NaN이었던 칸만 모델의 예측값으로 채우는 것.
#
# 판다스 where을 이용하면:
#   - 조건(~R_sparse_full.isna()) == True 인 곳 → R_sparse_full 값 유지
#   - 조건이 False(= NaN이었던 곳)           → R_hat_df_full 값으로 채움

completed_full = R_sparse_full.where(~R_sparse_full.isna(), R_hat_df_full)

print("원본 행렬 (일부)")
display(R_sparse_full.iloc[:5, :5])

print("MF로 NaN만 채운 행렬 (일부)")
display(completed_full.iloc[:5, :5].round(2))


원본 행렬 (일부)


movieId,1,2,3,4,5
userId,,,,,
1,5.0,3.0,4.0,3.0,3.0
2,4.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN


MF로 NaN만 채운 행렬 (일부)


movieId,1,2,3,4,5
userId,,,,,
1,5.00,3.00,4.00,3.00,3.00
2,4.00,3.41,3.40,3.52,3.28
3,3.24,3.01,2.93,3.39,2.58
4,4.31,4.04,3.70,3.55,4.06
5,4.00,3.00,2.90,3.18,2.47


---
이 아래는 VAE

In [ ]:
# ===== VAE 셀 1: train / val / test 유저×아이템 행렬 =====

import numpy as np
import torch

print("num_users:", num_users, "num_items:", num_items)

# -----------------------------------------------------------
# 1. (유저 수 × 아이템 수) 크기의 dense 행렬/마스크 틀 만들기
# -----------------------------------------------------------
#  - R_train : train 평점만 들어가는 행렬
#  - R_val   : val   평점만 들어가는 행렬
#  - R_test  : test  평점만 들어가는 행렬
#  - mask_*  : 해당 위치에 평점이 있는지(1) 없는지(0)를 나타내는 마스크
#
#  VAE 입장에서는 "한 유저의 모든 아이템 평점 벡터"를 입력으로 쓰기 때문에
#  이런 형태의 (num_users, num_items) 행렬이 필요하다.
#  값이 0인 곳은 "미관측"으로 취급하고, mask를 통해 손실 계산에서 제외할 것이다.
R_train = np.zeros((num_users, num_items), dtype=np.float32)
R_val   = np.zeros((num_users, num_items), dtype=np.float32)
R_test  = np.zeros((num_users, num_items), dtype=np.float32)

mask_train = np.zeros((num_users, num_items), dtype=np.float32)
mask_val   = np.zeros((num_users, num_items), dtype=np.float32)
mask_test  = np.zeros((num_users, num_items), dtype=np.float32)

# -----------------------------------------------------------
# 2. train_df의 rating을 R_train / mask_train에 채우기
# -----------------------------------------------------------
#  - train_df는 (user_idx, item_idx, rating)을 행 단위로 가지고 있다.
#  - 각 행(row)에 대해:
#      u: user_idx (0 ~ num_users-1)
#      i: item_idx (0 ~ num_items-1)
#      r: 실제 평점
#    R_train[u, i] 에 평점을 넣고
#    mask_train[u, i] = 1.0 으로 "여기는 관측된 평점"이라는 걸 표시한다.
for row in train_df.itertuples():
    u = row.user_idx
    i = row.item_idx
    r = row.rating
    R_train[u, i] = r
    mask_train[u, i] = 1.0

# -----------------------------------------------------------
# 3. val_df → R_val, mask_val
# -----------------------------------------------------------
#  - 검증용(val) 데이터도 구조는 train_df와 완전히 동일하다.
#  - 다만, 학습에는 사용하지 않고,
#    "모델이 얼마나 잘 일반화되는지"를 보기 위한 기준으로만 쓴다.
#  - 나중에 VAE는 항상 R_train을 입력으로 사용하고,
#    예측 결과를 val_df / test_df와 비교해서 RMSE를 계산한다.
for row in val_df.itertuples():
    u = row.user_idx
    i = row.item_idx
    r = row.rating
    R_val[u, i] = r
    mask_val[u, i] = 1.0

# -----------------------------------------------------------
# 4. test_df → R_test, mask_test
# -----------------------------------------------------------
#  - test_df는 "최종 성능"을 재는 용도.
#  - 학습/튜닝 과정에서는 절대로 건드리지 않고,
#    모든 모델 설정을 다 고른 뒤 마지막에 한 번만 사용한다.
for row in test_df.itertuples():
    u = row.user_idx
    i = row.item_idx
    r = row.rating
    R_test[u, i] = r
    mask_test[u, i] = 1.0

print("R_train shape:", R_train.shape)
print("train/val/test non-zeros:",
      mask_train.sum(), mask_val.sum(), mask_test.sum())
#  - R_train.shape : (num_users, num_items) 형태 확인
#  - mask_* .sum() : 각 split에 실제 평점이 몇 개 들어있는지(관측 수) 확인


num_users: 943 num_items: 1682
R_train shape: (943, 1682)
train/val/test non-zeros: 80000.0 10000.0 10000.0


In [ ]:
# ===== VAE 셀 2: Dataset / DataLoader (train 전용) =====

from torch.utils.data import Dataset, DataLoader

class UserRatingDataset(Dataset):
    """
    한 유저의 모든 아이템 평점 벡터를 하나의 샘플로 보는 Dataset.

    매 샘플은:
      x: 해당 유저의 rating 벡터  (길이 = num_items)
      m: 해당 유저의 mask 벡터   (관측=1, 미관측=0)

    - R: (num_users, num_items) rating 행렬 (0은 미관측)
    - M: (num_users, num_items) mask 행렬 (관측=1, 미관측=0)
    """
    def __init__(self, R, M):
        # numpy 배열을 torch 텐서로 바꿔서 저장
        #  R : 실제 평점 값 (미관측은 0)
        #  M : 어디가 관측되었는지 표시하는 마스크
        self.R = torch.from_numpy(R)  # (U, I)
        self.M = torch.from_numpy(M)  # (U, I)

    def __len__(self):
        # 전체 유저 수 (= 행 개수)
        return self.R.shape[0]

    def __getitem__(self, idx):
        # idx번째 유저의 (rating 벡터, mask 벡터) 한 쌍을 반환
        #  x: 해당 유저가 준 평점들 (미관측은 0)
        #  m: 그 위치가 관측된 평점인지 여부 (1/0)
        x = self.R[idx]  # (num_items,)
        m = self.M[idx]  # (num_items,)
        return x, m

# ----------------------------------------------------
# 학습용 Dataset / DataLoader
# ----------------------------------------------------
# VAE는 "유저 전체 × 아이템 전체 행렬"을 입력으로 쓰지만,
# 미니배치 학습을 위해 유저 단위로 잘라서 씀:
#   - train_user_dataset: 유저별 (rating, mask) 샘플 모음
#   - train_user_loader : 그걸 batch_size=64씩 섞어서 뽑아오는 DataLoader
#
# val/test 쪽은 Dataset을 따로 만들지 않고,
#   → 전체 R_train을 한 번에 VAE에 넣어서 재구성한 뒤
#   → 그 예측값을 val_df/test_df와 비교해서 RMSE를 구하는 방식으로 평가한다.
#     (그래서 여기서는 "train 전용" DataLoader만 있으면 충분함)

train_user_dataset = UserRatingDataset(R_train, mask_train)
train_user_loader  = DataLoader(train_user_dataset, batch_size=64, shuffle=True)

len(train_user_dataset)


943

In [ ]:
# ===== VAE 셀 3: VAE 모델 정의 =====

import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

class VAE(nn.Module):
    def __init__(self, num_items, hidden_dim=600, latent_dim=50):
        """
        num_items : 아이템(영화) 개수 = 입력/출력 벡터 길이
        hidden_dim: 인코더/디코더 중간 은닉층 크기
        latent_dim: 잠재공간(latent vector, z)의 차원

        구조:
          x (num_items) → [Encoder] → z (latent_dim) → [Decoder] → x_hat (num_items)
        """
        super().__init__()
        self.num_items = num_items

        # ----- Encoder -----
        # x: (batch_size, num_items)
        # 1) num_items → hidden_dim 비선형 변환
        self.fc1 = nn.Linear(*********, *********) #  입력 차원, 은닉 차원 (num_items, hidden_dim)


        # 2) hidden_dim → latent_dim
        #   VAE는 잠재 벡터 z를 확률분포 N(mu, sigma^2)로 놓기 때문에
        #   평균(mu)과 로그분산(logvar)을 각각 별도의 Linear로 뽑는다.
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)      # 평균 μ
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)  # 로그분산 log σ²

        # ----- Decoder -----
        # 잠재 벡터 z를 다시 원래 차원(num_items)으로 복원하는 MLP
        self.fc3 = nn.Linear(latent_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, num_items)

    # ----- 인코더: x → (mu, logvar) -----
    def encode(self, x):
        # x: (B, num_items)
        # 1) 은닉층 + ReLU
        h1 = F.relu(self.fc1(x))       # (B, hidden_dim)
        # 2) 평균과 로그분산을 각각 추출
        mu = self.fc_mu(h1)            # (B, latent_dim)
        logvar = self.fc_logvar(h1)    # (B, latent_dim)
        return mu, logvar

    # ----- 재파라미터화: (mu, logvar) → z 샘플링 -----
    def reparameterize(self, mu, logvar):
        """
        VAE 핵심: z ~ N(mu, sigma^2)를 직접 샘플링하면
        역전파가 안 되므로,
        z = mu + sigma * eps,  eps ~ N(0, I)
        형태로 바꿔서 gradient가 mu, logvar까지 흘러가게 만든다.
        """
        std = torch.exp(0.5 * logvar)  # logvar = log(sigma^2) → sigma = exp(0.5 * logvar)
        eps = torch.randn_like(std)    # mu와 같은 shape의 표준정규 노이즈 N(0, I)
        return mu + eps * std          # (B, latent_dim)

    # ----- 디코더: z → 복원된 평점 벡터 -----
    def decode(self, z):
        # z: (B, latent_dim)
        h3 = F.relu(self.fc3(z))       # (B, hidden_dim)
        # 출력: (B, num_items)
        # 여기서는 별도의 활성함수 없이 "실수값"으로 그대로 둔다.
        # (이후 loss에서 MSE 기반 reconstruction을 쓸 것이기 때문)
        return self.fc4(h3)

    # ----- 전체 forward: x → (recon_x, mu, logvar) -----
    def forward(self, x):
        """
        x: (batch_size, num_items)
        반환:
          recon : 복원된 평점 벡터 (B, num_items)
          mu, logvar : 잠재 분포 파라미터 (둘 다 (B, latent_dim))
        """
        mu, logvar = self.*******(x) #  encode 호출
        z = self.**************(mu, logvar) #  reparameterize 호출
        recon = self.*******(z)  #  decode 호출
        return recon, mu, logvar

# VAE 모델 인스턴스 생성
vae = VAE(num_items=num_items, hidden_dim=600, latent_dim=50).to(device)
vae


device: cuda


VAE(
  (fc1): Linear(in_features=1682, out_features=600, bias=True)
  (fc_mu): Linear(in_features=600, out_features=50, bias=True)
  (fc_logvar): Linear(in_features=600, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=600, bias=True)
  (fc4): Linear(in_features=600, out_features=1682, bias=True)
)

In [ ]:
# ===== VAE 셀 4: 손실 함수 + 옵티마이저 =====

def vae_loss(recon_x, x, mu, logvar, mask, beta=0.001):
    """
    VAE의 총 손실 = 재구성 오차(reconstruction) + β * KL divergence

    매개변수
    ----------
    recon_x : (B, num_items)
        디코더가 복원한 평점 벡터 (예측값)
    x       : (B, num_items)
        입력으로 들어온 실제 평점 벡터 (없는 곳은 0)
    mu      : (B, latent_dim)
        인코더가 뽑은 잠재 분포의 평균
    logvar  : (B, latent_dim)
        인코더가 뽑은 잠재 분포의 로그분산
    mask    : (B, num_items)
        관측된 평점 위치만 1, 나머지 0
        → "없는 칸"은 reconstruction loss에서 빼기 위해 사용
    beta    : float
        KL 항의 가중치 (β-VAE에서 쓰는 β)

    반환
    ----------
    total_loss : 전체 손실 (scalar)
    recon_loss : 재구성 오차 (detach된 값, 로깅용)
    kl_loss    : KL 오차 (detach된 값, 로깅용)
    """

    # ----- 1. 재구성 오차 (관측된 칸만 사용) -----
    # diff2: (예측 - 실제)^2, 모든 아이템에 대해
    diff2 = (recon_x - x) ** 2  # (B, I)

    # diff2 * mask:
    #   - 관측된 칸(mask=1)만 남기고
    #   - 관측 안 된 칸(mask=0)은 0으로 만들어 버림
    # sum(dim=1):
    #   - 각 유저별로 "관측된 아이템들에 대한 제곱오차 합"을 구함
    #
    # mask.sum(dim=1):
    #   - 유저별로 "관측된 아이템 개수"
    #
    # → per_user_mse:
    #   - 각 유저에 대해 "관측된 칸들에 대한 MSE"
    #   - eps는 나눗셈 0 방지용 (관측 아이템이 0개인 유저가 있을 수 있어서)
    eps = 1e-8
    per_user_mse = (diff2 * mask).sum(dim=1) / (mask.sum(dim=1) + eps)  # (B,)

    # 여러 유저의 MSE를 평균내서 전체 reconstruction loss로 사용
    recon_loss = **************.mean()  # scalar #  여러 유저의 MSE를 평균 (per_user_mse)

    # ----- 2. KL divergence (잠재 분포 vs 표준 정규분포) -----
    # VAE 논문에서 나온 공식:
    #   KL( N(mu, sigma^2) || N(0, I) )
    #   = -0.5 * Σ (1 + logσ^2 - μ^2 - σ^2)
    #
    # 여기서 logvar = log σ^2,
    # logvar.exp() = σ^2
    #
    # dim=1로 합을 구하면, 각 샘플(유저)별 KL 값을 얻음 → 그걸 평균냄
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)  # (B,)
    kl_loss = kl.mean()  # scalar

    # ----- 3. 총 손실 = 재구성 오차 + β * KL -----
    # beta를 0.001처럼 작게 두면:
    #   - "재구성 정확도"를 더 우선시하면서
    #   - latent space가 너무 제멋대로 퍼지지 않도록 살짝 규제
    total_loss = recon_loss + beta * kl_loss

    # 학습에만 total_loss를 쓰고,
    # recon_loss와 kl_loss는 그래프에서 분리(detach)해서 로깅용으로만 반환
    return total_loss, recon_loss.*****(), kl_loss.*****()


# ----- 4. VAE용 옵티마이저 설정 -----
# Adam:
#   - SGD보다 학습률을 자동으로 조정해주는 optimizer
#   - VAE 같이 꽤 깊은 네트워크에도 안정적으로 잘 작동
optimizer_vae = torch.optim.Adam(
    vae.parameters(),
    lr=0.001,        # VAE 학습률
    weight_decay=1e-6  # L2 정규화 (너무 큰 가중치 방지, 약한 정규화)
)


In [ ]:
# ===== VAE 셀 5: 학습 루프 + val 기반 Early Stopping + 최종 test RMSE =====

from math import sqrt

def get_vae_predictions(vae, R_input):
    """
    R_input: numpy array (num_users, num_items)
             - 보통 R_train (train 평점만 채워둔 행렬)을 넣는다.
             - 이유: VAE는 "train에서 본 패턴"을 이용해서
                     유저 × 아이템 전체에 대한 예측을 만들기 때문.

    반환값:
      R_pred: torch tensor (num_users, num_items), CPU에 올라간 예측 행렬
              각 원소는 "해당 유저가 그 아이템을 얼마나 줄 것 같은지"를 의미.
    """
    vae.eval()  # 평가 모드 (dropout, batchnorm 등 비활성화)
    with torch.no_grad():
        X = torch.from_numpy(R_input).to(device)  # (U, I)
        batch_size = 128
        preds = []

        # 유저 수가 많으니, 한 번에 다 넣지 말고 batch 단위로 나누어 처리
        for start in range(0, X.size(0), batch_size):
            end = start + batch_size
            x_batch = X[start:end]           # (B, I)
            mu, logvar = vae.encode(x_batch) # 인코더로 잠재표현 추출
            # 평가 시에는 샘플링 대신 z=mu를 그대로 사용 (variance 제거)
            z = mu
            recon_batch = vae.decode(z)      # (B, I) 복원 벡터
            preds.append(recon_batch.cpu())  # CPU로 옮겨서 리스트에 저장

        # 배치별 결과를 유저 축(U) 방향으로 이어붙여 전체 예측 행렬 구성
        R_pred = torch.cat(preds, dim=0)     # (U, I)

    return R_pred  # CPU tensor


def evaluate_vae_rmse(vae, R_input, df_eval):
    """
    VAE 예측 vs df_eval(train/val/test 중 하나)의 실제 rating으로 RMSE 계산.

    - vae     : 학습된 VAE 모델
    - R_input : 예측에 사용할 입력 행렬 → 주로 R_train
                (train에서 본 정보만 가지고 전체 유저×아이템을 복원하게 함)
    - df_eval : 평가용 DataFrame (val_df 또는 test_df)
                컬럼: user_idx, item_idx, rating

    흐름:
      1) R_input을 넣어 전체 예측 행렬 R_pred (U×I)를 얻고
      2) df_eval에 있는 (u, i) 위치만 골라
         실제 평점 vs 예측 평점을 비교해 RMSE를 계산한다.
    """
    R_pred = get_vae_predictions(vae, R_input)  # (U, I)
    se = 0.0
    cnt = 0
    with torch.no_grad():
        for row in df_eval.itertuples():
            u = row.user_idx
            i = row.item_idx
            true_r = row.rating              # 실제 평점
            pred_r = R_pred[u, i].item()     # VAE 예측 평점

            # MovieLens 평점 범위(1~5)에 맞게 잘라서 평가
            pred_r = max(1.0, min(5.0, pred_r))

            se += (pred_r - true_r) ** 2     # 제곱오차 누적
            cnt += 1

    return sqrt(se / cnt)                    # RMSE 반환


# -------- 학습 루프 --------
n_epochs = 50

best_val = float("inf")  # 지금까지 본 val RMSE 중 최소값
best_epoch = None        # 최소값이 나왔던 epoch 번호
best_state = None        # 그때의 모델 파라미터(state_dict)

for epoch in range(1, n_epochs + 1):
    vae.train()
    total_loss = 0.0
    total_recon = 0.0
    total_kl = 0.0
    num_batches = 0

    # ----------------------------------------------------
    # 1) train_user_loader로 한 epoch 학습
    #    - 입력: "train에서 관측된 rating만 들어 있는 R_train"
    #    - 목표: 관측된 칸(mask=1)에 대해서만 잘 복원하도록 학습
    # ----------------------------------------------------
    for x, m in train_user_loader:
        x = x.to(device)  # (B, I) 유저별 rating 벡터
        m = m.to(device)  # (B, I) 유저별 mask 벡터

        optimizer_vae.zero_grad()
        recon_x, mu, logvar = vae(x)
        # vae_loss 안에서:
        #   - recon_loss: 관측된 칸만 골라 MSE (per-user 평균)
        #   - kl_loss   : 인코더 분포 q(z|x)와 N(0, I)의 KL divergence
        loss, recon_l, kl_l = vae_loss(
            recon_x, x, mu, logvar, m, beta=0.001
        )
        loss.backward()
        optimizer_vae.step()

        total_loss += loss.item()
        total_recon += recon_l.mean().item()
        total_kl += kl_l.mean().item()
        num_batches += 1

    avg_loss = total_loss / num_batches
    avg_recon = total_recon / num_batches
    avg_kl = total_kl / num_batches

    # ----------------------------------------------------
    # 2) val RMSE 계산 (모델 파라미터 업데이트는 하지 않음)
    # ----------------------------------------------------
    #   - 입력으로는 여전히 R_train을 사용한다.
    #   - 복원된 전체 행렬 R_pred에서,
    #     val_df에 있는 (user_idx, item_idx) 위치만 골라 평가.
    val_rmse = evaluate_vae_rmse(vae, R_train, val_df)

    print(
        f"Epoch {epoch:03d} | loss: {avg_loss:.4f} "
        f"(recon: {avg_recon:.4f}, kl: {avg_kl:.4f}) | val RMSE: {val_rmse:.4f}"
    )

    # ----------------------------------------------------
    # 3) Early Stopping: val 성능이 가장 좋았던 시점의 모델만 기억
    # ----------------------------------------------------
    if val_rmse < best_val:
        best_val = val_rmse
        best_epoch = epoch
        # 현재 state_dict를 CPU로 복사해서 저장 (이후 rollback용)
        best_state = {k: v.cpu().clone() for k, v in vae.state_dict().items()}

print(f"\nBest VAE val RMSE: {best_val:.4f} at epoch {best_epoch}")

# --------------------------------------------------------
# 4) 학습이 끝나면, "가장 val이 좋았던 epoch"의 파라미터로 롤백
#    → 이후에 이 vae를 사용할 때는 항상 best 상태로 쓰게 된다.
# --------------------------------------------------------
vae.load_state_dict(best_state)

# --------------------------------------------------------
# 5) 최종 test RMSE는 여기서 딱 한 번만 계산
#    - test_df는 지금까지 학습/튜닝에 전혀 사용되지 않았고,
#      오로지 "최종 성능 측정" 용도로만 쓰인다.
# --------------------------------------------------------
test_rmse = evaluate_vae_rmse(vae, R_train, test_df)
print(f"Final VAE test RMSE: {test_rmse:.4f}")

Epoch 001 | loss: 6.7489 (recon: 6.6580, kl: 90.8570) | val RMSE: 1.6585
Epoch 002 | loss: 1.9802 (recon: 1.9333, kl: 46.8145) | val RMSE: 1.4692
Epoch 003 | loss: 1.3121 (recon: 1.2821, kl: 29.9897) | val RMSE: 1.2837
Epoch 004 | loss: 1.1630 (recon: 1.1334, kl: 29.6575) | val RMSE: 1.1411
Epoch 005 | loss: 1.0894 (recon: 1.0562, kl: 33.2140) | val RMSE: 1.1348
Epoch 006 | loss: 1.0376 (recon: 1.0008, kl: 36.8244) | val RMSE: 1.1054
Epoch 007 | loss: 1.0115 (recon: 0.9717, kl: 39.8590) | val RMSE: 1.0989
Epoch 008 | loss: 0.9806 (recon: 0.9396, kl: 41.0151) | val RMSE: 1.0625
Epoch 009 | loss: 0.9676 (recon: 0.9255, kl: 42.1733) | val RMSE: 1.0488
Epoch 010 | loss: 0.9363 (recon: 0.8937, kl: 42.5545) | val RMSE: 1.0217
Epoch 011 | loss: 0.9170 (recon: 0.8729, kl: 44.1090) | val RMSE: 1.0074
Epoch 012 | loss: 0.8878 (recon: 0.8418, kl: 45.9878) | val RMSE: 0.9999
Epoch 013 | loss: 0.8567 (recon: 0.8096, kl: 47.1134) | val RMSE: 1.0008
Epoch 014 | loss: 0.8345 (recon: 0.7862, kl: 48.264

In [ ]:
# ===== VAE로 NaN만 채운 평점 행렬 만들기 =====

import pandas as pd
import torch

# 1) userId × movieId 원본 평점 행렬 (NaN 포함) 다시 만들기
#    - MovieLens rating 데이터에서
#      행: userId, 열: movieId
#      값: 실제 rating (없는 곳은 NaN) 형태의 pivot table
R_sparse_full = ratings.pivot_table(
    values="rating",
    index="userId",
    columns="movieId"
)

print("원본 행렬 (일부)")
display(R_sparse_full.iloc[:5, :5])  # 상위 5×5만 맛보기


# 2) VAE로 전체 유저 × 아이템 예측 행렬 얻기
#    - R_train(유저×아이템, 없는 곳은 0) 을 입력으로 넣고
#    - get_vae_predictions를 통해 (num_users, num_items) 예측행렬을 얻는다.
with torch.no_grad():
    # R_pred: (num_users, num_items) CPU tensor
    R_pred = ***************(vae, R_train)

    # 예측값도 1~5 범위로 자르기 (실제 평점 스케일 맞추기)
    R_pred = torch.clamp(R_pred, 1.0, 5.0)

    # 나중에 DataFrame으로 만들기 위해 numpy로 변환
    R_pred_np = R_pred.*****()


# 3) 예측 행렬을 userId / movieId 축을 가진 DataFrame으로 변환
#    - 행 인덱스를 user_ids (정렬된 userId 리스트)
#    - 열 인덱스를 movie_ids (정렬된 movieId 리스트)
#    로 지정해서,
#    원래 pivot_table과 같은 축을 가지도록 맞춰준다.
R_pred_df = pd.DataFrame(
    R_pred_np,
    index=user_ids,    # 행: userId
    columns=movie_ids  # 열: movieId
)

print("VAE 예측 행렬 (일부)")
display(R_pred_df.iloc[:5, :5].round(2))


# 4) 원래 값이 있는 칸은 그대로 두고, NaN이었던 칸만 VAE 예측으로 채우기
#
#   - R_sparse_full: 실제 관측 평점 (NaN 포함)
#   - R_pred_df: VAE가 예측한 "모든 유저×아이템" 평점
#
#   DataFrame.where(조건, 다른값):
#     - 조건이 True인 칸은 원본 값을 유지
#     - 조건이 False인 칸만 다른값으로 교체
#
#   여기서 조건은 "~R_sparse_full.isna()" (NaN이 아닌 곳만 True)
#   → 원래 rating이 있는 칸은 R_sparse_full 그대로 두고
#   → NaN인 칸만 R_pred_df 값으로 채워 넣는다.
completed_vae = R_sparse_full.where(~R_sparse_full.isna(), *********** # 👉 NaN이던 곳(False)에 채울 예측값 (R_pred_df)

print("VAE로 NaN만 채운 완성 행렬 (일부)")
display(completed_vae.iloc[:5, :5].round(2))

원본 행렬 (일부)


movieId,1,2,3,4,5
userId,,,,,
1,5.0,3.0,4.0,3.0,3.0
2,4.0,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN


VAE 예측 행렬 (일부)


,1,2,3,4,5
1,4.29,3.30,3.04,4.02,2.64
2,3.71,2.64,1.91,3.49,3.37
3,2.87,2.20,2.13,2.88,2.71
4,4.46,4.07,4.16,4.81,4.15
5,3.80,2.82,2.56,3.46,2.64


VAE로 NaN만 채운 완성 행렬 (일부)


movieId,1,2,3,4,5
userId,,,,,
1,5.00,3.00,4.00,3.00,3.00
2,4.00,2.64,1.91,3.49,3.37
3,2.87,2.20,2.13,2.88,2.71
4,4.46,4.07,4.16,4.81,4.15
5,4.00,3.00,2.56,3.46,2.64
